In [4]:
import pandas as pd 
from nltk.corpus import stopwords
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import numpy as np
from numpy import argmax
from torch.nn.functional import softmax

In [5]:
model_name = "ProsusAI/finbert"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
df = pd.read_csv('FinalStockNews.csv')
for col in df.columns:
    col = col.strip
df.drop('Unnamed: 0', axis=1, inplace=True)
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)
df

,Unnamed: 0.1,ticker,title,date,link,articleInfo,paragraphList
0,0,AMZN,"Tech Layoffs, Remote Work Push Office Vacancie...",2024-04-16T00:00:00,/news/etf/tech-layoffs-remote-work-push-office...,The commercial real estate industry continues ...,['The commercial real estate industry continue...
1,1,AMZN,The 3 Best Stocks to Buy With Your Tax Refund ...,2024-04-16T00:00:00,/news/stocks/the-3-best-stocks-to-buy-with-you...,The IRS typically gives tax refunds within 21 ...,"['InvestorPlace - Stock Market News, Stock Adv..."
2,2,AMZN,3 AI Stocks to Buy Now: Q2 Edition,2024-04-16T00:00:00,/news/stocks/3-ai-stocks-to-buy-now-q2-edition...,Just like electricity and the internet unleash...,"['InvestorPlace - Stock Market News, Stock Adv..."
3,3,AMZN,"Amazon, Meta, Uber Remain 'Top Overall Picks' ...",2024-04-16T00:00:00,/news/stocks/amazon-meta-uber-remain-top-overa...,As the first-quarter (Q1) 2024 earnings season...,['As the first-quarter (Q1) 2024 earnings seas...
4,4,AMZN,Amazon (AMZN) Receives a Buy from J.P. Morgan,2024-04-16T00:00:00,/news/stocks/amazon-amzn-receives-a-buy-from-j...,J.P. Morgan analyst Doug Anmuth maintained a B...,['J.P. Morgan analyst Doug Anmuth maintained a...
...,...,...,...,...,...,...,...
1559,1559,AAPL,Riding the Bull: 3 ETFs to Capitalize on the O...,2024-03-13T00:00:00,/news/stocks/riding-the-bull-3-etfs-to-capital...,There are many advantages to owing exchange-tr...,"['InvestorPlace - Stock Market News, Stock Adv..."
1560,1560,AAPL,Apple’s Bitter Bite: Is the Tech Giant’s Slowi...,2024-03-13T00:00:00,/news/stocks/apples-bitter-bite-is-the-tech-gi...,The market has shown extreme volatility for mo...,"['InvestorPlace - Stock Market News, Stock Adv..."
1561,1561,AAPL,SCHD ETF Spotlight: The Top 6 Stocks Inside Th...,2024-03-13T00:00:00,/news/stocks/schd-etf-spotlight-the-top-6-stoc...,Investing in the stock market is one of the su...,"['InvestorPlace - Stock Market News, Stock Adv..."
1562,1562,AAPL,Maintaining Market-Perform on Apple: A Balance...,2024-03-13T00:00:00,/news/stocks/maintaining-market-perform-on-app...,Bernstein analyst Toni Sacconaghi maintained a...,['Bernstein analyst Toni Sacconaghi maintained...


In [7]:
def getDfStock(ticker, startDate, endDate):
    # Convert startDate and endDate to datetime objects to ensure proper comparison
    start_date = pd.to_datetime(startDate)
    end_date = pd.to_datetime(endDate)

    # Ensure the 'date' column is in datetime format
    df['date'] = pd.to_datetime(df['date'])

    # Filter the DataFrame for the given ticker and the date range inclusive
    filtered_df = df[(df['ticker'] == ticker) & (df['date'] >= start_date) & (df['date'] <= end_date)]

    return filtered_df    

In [8]:
# checking validity of getDfStock
dfNew= getDfStock('AAPL','2024-03-01','2024-03-15')
dfNew

,Unnamed: 0.1,ticker,title,date,link,articleInfo,paragraphList
1536,1536,AAPL,QQQI: The Undiscovered Nasdaq-100 Covered Call...,2024-03-15,/news/etf/qqqi-the-undiscovered-nasdaq-100-cov...,"A few months ago, I analyzed the total perform...","['A few months ago, I analyzed the total perfo..."
1537,1537,AAPL,The 3 Most Undervalued Large-Cap Stocks to Buy...,2024-03-15,/news/stocks/the-3-most-undervalued-large-cap-...,Value investing has long been a popular and su...,"['InvestorPlace - Stock Market News, Stock Adv..."
1538,1538,AAPL,Apple Acquires Canadian Startup DarwinAI: Repo...,2024-03-15,/news/stocks/apple-acquires-canadian-startup-d...,(RTTNews) - Tech giant Apple Inc.(AAPL) has re...,['(RTTNews) - Tech giant Apple Inc.(AAPL) has ...
1539,1539,AAPL,Nvidia Blazes Trail For 3 Further Phases Of AI...,2024-03-15,/news/stocks/nvidia-blazes-trail-for-3-further...,NVIDIA Corp (NASDAQ:NVDA) blazed the trail for...,['NVIDIA Corp (NASDAQ:NVDA) blazed the trail f...
1540,1540,AAPL,"Old Dogs, New Tricks: 3 Dow Stocks Poised for ...",2024-03-15,/news/stocks/old-dogs-new-tricks-3-dow-stocks-...,"Although the S&P 500 is at all-time highs, som...","['InvestorPlace - Stock Market News, Stock Adv..."
1541,1541,AAPL,Don’t Fight the Tide: Nvidia’s AI Momentum Is ...,2024-03-15,/news/stocks/dont-fight-the-tide-nvidias-ai-mo...,Nvidia (NASDAQ:NVDA) has been on an absolute t...,"['InvestorPlace - Stock Market News, Stock Adv..."
1542,1542,AAPL,'This Is A Long-Term Bull Market': Expert Ed Y...,2024-03-15,/news/stocks/this-is-a-long-term-bull-market-e...,"Ed Yardeni, the renowned economist and market ...","['Ed Yardeni, the renowned economist and marke..."
1543,1543,AAPL,Buy Rating for Apple Stock: Anticipated Growth...,2024-03-14,/news/stocks/buy-rating-for-apple-stock-antici...,Analyst Atif Malik of Citi maintained a Buy ra...,['Analyst Atif Malik of Citi maintained a Buy ...
1544,1544,AAPL,Tesla Worst Performing S&P 500 Stock Year-To-D...,2024-03-14,/news/stocks/tesla-worst-performing-s-p-500-st...,Electric vehicle leader Tesla Inc (NASDAQ:TSLA...,['Electric vehicle leader Tesla Inc (NASDAQ:TS...
1545,1545,AAPL,Apple Vision Pro's Code Hints At Potential Lau...,2024-03-14,/news/stocks/apple-vision-pro-s-code-hints-at-...,(RTTNews) - Tech giant Apple's (AAPL) sensatio...,"[""(RTTNews) - Tech giant Apple's (AAPL) sensat..."


In [9]:
def sliding_window(text, tokenizer, max_length=512, overlap=50):

    tokenized_text = tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True)
    input_ids = tokenized_text['input_ids'].squeeze().numpy().tolist()
    
    total_length = len(input_ids)
    step_size = max_length - overlap
    chunks = []
    
    for start in range(0, total_length, step_size):
        end = start + max_length
        chunk = input_ids[start:end]
        
        chunk = [tokenizer.cls_token_id] + chunk + [tokenizer.sep_token_id]
        chunks.append(chunk)
    
    return chunks


In [24]:
# unit test sliding window
assert sliding_window("This is the test", tokenizer, 2, 1) == [[101, 101, 2023, 102],
 [101, 2023, 2003, 102],
 [101, 2003, 1996, 102],
 [101, 1996, 3231, 102],
 [101, 3231, 102, 102],
 [101, 102, 102]]
assert sliding_window("The fox jumped over the lake and tackled Tom Brady", tokenizer, 6, 3) == [[101, 101, 1996, 4419, 5598, 2058, 1996, 102],
 [101, 5598, 2058, 1996, 2697, 1998, 26176, 102],
 [101, 2697, 1998, 26176, 3419, 10184, 102, 102],
 [101, 3419, 10184, 102, 102]]

In [10]:
def process_chunks(chunks, tokenizer, model):
    model.eval()  
    sentiments = []
    scores = []

    with torch.no_grad():
        for chunk in chunks:
            inputs = torch.tensor(chunk).unsqueeze(0)  
            

            inputs = torch.nn.functional.pad(inputs, (0, 512 - inputs.shape[1]), value=tokenizer.pad_token_id)
            
            outputs = model(inputs)
            logits = outputs.logits
            sentiment = torch.argmax(logits, dim=1).numpy()[0]  
            score = torch.softmax(logits, dim=1).max().item()  

            sentiments.append(sentiment)
            scores.append(score)

    return sentiments, scores


In [28]:
# unit tests for process_chunks
chunk1 = [[101, 101, 2023, 102],
 [101, 2023, 2003, 102],
 [101, 2003, 1996, 102],
 [101, 1996, 3231, 102],
 [101, 3231, 102, 102],
 [101, 102, 102]]
assert process_chunks(chunk1, tokenizer, model) == ([2, 2, 2, 2, 2, 2],
 [0.7957010269165039,
  0.8266777992248535,
  0.7992331385612488,
  0.7978023886680603,
  0.7663531303405762,
  0.7840284109115601])
chunk2 = [[101, 101, 1996, 4419, 5598, 2058, 1996, 102],
 [101, 5598, 2058, 1996, 2697, 1998, 26176, 102],
 [101, 2697, 1998, 26176, 3419, 10184, 102, 102],
 [101, 3419, 10184, 102, 102]]
assert process_chunks(chunk2, tokenizer, model) == ([2, 2, 2, 2],
 [0.7723026871681213,
  0.7744370698928833,
  0.7692530155181885,
  0.7506882548332214])

In [11]:
def aggregate_results(sentiments, scores):

    average_score_per_sentiment = {}
    for sentiment, score in zip(sentiments, scores):
        if sentiment not in average_score_per_sentiment:
            average_score_per_sentiment[sentiment] = []
        average_score_per_sentiment[sentiment].append(score)
    
    for sentiment in average_score_per_sentiment:
        average_score_per_sentiment[sentiment] = sum(average_score_per_sentiment[sentiment]) / len(average_score_per_sentiment[sentiment])
    
    dominant_sentiment = max(average_score_per_sentiment, key=average_score_per_sentiment.get)
    return dominant_sentiment, average_score_per_sentiment[dominant_sentiment]


In [31]:
# unit tests for aggregate_results
sent1 = [2, 2, 2, 2, 2, 2]
score1 = [0.7957010269165039,
  0.8266777992248535,
  0.7992331385612488,
  0.7978023886680603,
  0.7663531303405762,
  0.7840284109115601]
assert aggregate_results(sent1, score1) == (2, 0.7949659824371338)

sent2 = [2, 2, 2, 2, 2, 2]
score2 = [0.7957010269165039,
  0.8266777992248535,
  0.7992331385612488,
  0.7978023886680603,
  0.7663531303405762,
  0.7840284109115601]
assert aggregate_results(sent2, score2) == (2, 0.7949659824371338)

In [12]:
def full_window(text, tokenizer, model):
    chunks = sliding_window(text, tokenizer)
    sentiments, scores = process_chunks(chunks, tokenizer, model)
    return aggregate_results(sentiments, scores)

In [35]:
# unit test full_window
text1 = 'Contrary to popular belief, Lorem Ipsum is not simply random text. It has roots in a piece of classical Latin literature from 45 BC, making it over 2000 years old. Richard McClintock, a Latin professor at Hampden-Sydney College in Virginia, looked up one of the more obscure Latin words, consectetur, from a Lorem Ipsum passage, and going through the cites of the word in classical literature, discovered the undoubtable source. Lorem Ipsum comes from sections 1.10.32 and 1.10.33 of "de Finibus Bonorum et Malorum" (The Extremes of Good and Evil) by Cicero, written in 45 BC. This book is a treatise on the theory of ethics, very popular during the Renaissance. The first line of Lorem Ipsum, "Lorem ipsum dolor sit amet..", comes from a line in section 1.10.32.'
assert full_window(text1, tokenizer, model) == (2, 0.8298267126083374)

text2 = "It is a long established fact that a reader will be distracted by the readable content of a page when looking at its layout."
assert full_window(text2, tokenizer, model) == (2, 0.5958979725837708)

In [13]:
def getSentimentScore(ticker, startDate, endDate):
    # Retrieve the subset DataFrame for the specified ticker and date range
    subDf = getDfStock(ticker, startDate, endDate)

    # Assume the function `full_window` applies sentiment analysis and returns the sentiment as 'positive', 'negative', or 'neutral'
    subDf['sentiment'] = subDf['articleInfo'].apply(lambda articleInfo: full_window(articleInfo, tokenizer, model)[0])

    # Map the sentiment results to numeric values: positive -> 1, neutral -> 0.5, negative -> 0
    sentiment_scores = subDf['sentiment'].map({'positive': 1, 'neutral': 0.5, 'negative': 0})

    # Calculate the average sentiment score
    if len(sentiment_scores) > 0:
        average_sentiment_score = sentiment_scores.mean()
    else:
        average_sentiment_score = None  # Handle case where no articles were found

    return average_sentiment_score


In [38]:
# unit test for getSentimentScore
getSentimentScore('AMZN', '2024-03-12', '2024-03-15')

/var/folders/2f/kkjcq0_90x7dmw3vb17_ggp40000gp/T/ipykernel_24842/1631622798.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subDf['sentiment'] = subDf['articleInfo'].apply(lambda articleInfo: full_window(articleInfo, tokenizer, model)[0])


nan

In [14]:
df['sentiment'] = df['articleInfo'].apply(lambda news: full_window(news, tokenizer, model)[0])
df['sentiment probability']  = df['articleInfo'].apply(lambda news: full_window(news, tokenizer, model)[1])

Token indices sequence length is longer than the specified maximum sequence length for this model (935 > 512). Running this sequence through the model will result in indexing errors
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [15]:
df['sentiment'].value_counts()

sentiment
2    774
0    485
1    305
Name: count, dtype: int64

In [16]:
full_window(df['articleInfo'][0], tokenizer, model)

(1, 0.941900759935379)